In [7]:
import asyncio
import pickle

import pandas as pd
%reload_ext autoreload
%autoreload 2
%autoreload now
import logging

from data_collection.generate_snapshot import generate_snapshot
from data_collection.models import Game, Round
from data_collection.questions_candidates import get_question_candidates
from data_collection.questions_pick import pick_question
from data_collection.semaphore import set_semaphore
from data_collection.guess import guess
from data_collection.answer import answer
from data_collection.simulate_game import run_game


logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

from dotenv import load_dotenv
from openai import AsyncOpenAI

from data_collection.utils import call_gpt4, dataset

load_dotenv()

True

In [2]:
set_semaphore("gpt-4", 100)

In [3]:
client = AsyncOpenAI()

In [4]:
# {
#         "things": 100,
#         "city": 40,
#         "country": 20,
#         "landmark": 40,
#     }

In [5]:
keywords_by_category = pickle.load(open("../keywords_by_category.pkl", "rb"))
all_dfs = pd.concat(keywords_by_category.values())
keywords_flat = list(all_dfs.keyword)

In [8]:
# generated = await generate_snapshot(client, "ice tea")
all_generated = await asyncio.gather(*[generate_snapshot(client, keyword) for keyword in keywords_flat])

2024-07-09 14:40:17,312 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-07-09 14:40:17,396 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-07-09 14:40:17,455 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-07-09 14:40:17,650 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-07-09 14:40:17,961 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-07-09 14:40:18,078 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-07-09 14:40:18,095 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-07-09 14:40:18,138 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-07-09 14:40:18,156 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "

In [10]:
# all_generated is list of lists
# Concat them into a flat list
# total_generated = []
# for generated in all_generated:
#     total_generated.extend(generated)
    
total_generated = [generated for generated_list in all_generated for generated in generated_list]

In [19]:
pickle.dump(total_generated, open("dataset_17476_from_snapshots.pkl", "wb"))

In [11]:
len(total_generated)

17476

In [54]:
total_generated[1]

{'prompt': 'You are playing "20 Questions" game.\n    \nHere are the previous questions and answers:\nQuestion: Is it a place? Answer: yes Your guess: Paris\n    \nYou need to generate 1 candidates for the next question to ask.\nYou should mix safe questions with more creative and non-orthodox ones.\nSafe questions are usually ones that split the space of possibilities in half.\nIf you have a few questions left, you can start acting more aggressively, intuitively checking some less probable, but high-reward hypotheses.\nConsider asking non-standard questions like "Does the keyword consist of a single word?, however only do it if you don\'t know it yet, and if you think that it\'s important to know at this point in the game.\nConsider asking questions like "Does the keyword start with letter \'A\'?" or "Does the keyword contain letter \'E\'?". However, don\'t overdo it, focus on more substantial questions.\n\nFormat your answer like this: output every question on a new line. Don\'t outp

In [46]:
for g in total_generated:
    if g['type'] == "guess_from_snapshot":
        keyword = g['response'].split("\n")[-1].strip()
        prompt_lines = g['prompt'].split("\n")
        # Find indexes of line that contain keyword:
        keyword_indexes = [i for i, line in enumerate(prompt_lines) if f"Your guess: {keyword}".lower().strip() in line.lower()]
        if not keyword_indexes and keyword.endswith("s"):
            keyword_indexes = [i for i, line in enumerate(prompt_lines) if f"Your guess: {keyword[:-1]}".lower().strip() in line.lower()]
        if keyword_indexes:
            print(g['prompt'])
            print("!!!!!!!!", keyword)
        # g['prompt'] = "\n".join([line for i, line in enumerate(prompt_lines) if i not in keyword_indexes])

In [59]:
for g in total_generated:
    if g['type'] == "questions_candidates_from_snapshot":
        new_response = g['response'].split("Question: ")[1].split("Answer")[0].strip()
        g['response'] = new_response

In [60]:
pickle.dump(total_generated, open("dataset_17476_from_snapshots_fixed.pkl", "wb"))

In [52]:
total_generated[504]

{'prompt': 'You are playing "20 Questions" game.\n    \nHere are the previous questions and answers:\nQuestion: Is it a place? Answer: yes Your guess: Paris\n    \nYou need to generate 1 candidates for the next question to ask.\nYou should mix safe questions with more creative and non-orthodox ones.\nSafe questions are usually ones that split the space of possibilities in half.\nIf you have a few questions left, you can start acting more aggressively, intuitively checking some less probable, but high-reward hypotheses.\nConsider asking non-standard questions like "Does the keyword consist of a single word?, however only do it if you don\'t know it yet, and if you think that it\'s important to know at this point in the game.\nConsider asking questions like "Does the keyword start with letter \'A\'?" or "Does the keyword contain letter \'E\'?". However, don\'t overdo it, focus on more substantial questions.\n\nFormat your answer like this: output every question on a new line. Don\'t outp

In [7]:
game = await run_game(client, "headphones")

Game headphones, round 0


2024-07-05 23:19:49,556 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Tokens used: 540


2024-07-05 23:19:51,194 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Tokens used: 718


...

Tokens used: 1024


2024-07-05 23:21:47,271 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Tokens used: 708


2024-07-05 23:21:48,852 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Tokens used: 1097


In [8]:
print(dataset[-1]['prompt'])

You are playing "20 Questions" game.
    
Here are the previous questions and answers:
    Question: Is it a living thing? Answer: no Your guess: is it a place
Question: Is it man-made? Answer: yes Your guess: eiffel tower
Question: Is it commonly found in a household? Answer: yes Your guess: television
Question: Is it electronic? Answer: yes Your guess: microwave oven
Question: The best next question is: Does it have a screen? Answer: no Your guess: refrigerator
Question: Is it used in the kitchen? Answer: no Your guess: vacuum cleaner
Question: Is it used for cleaning? Answer: no Your guess: electric fan
Question: Is it used for entertainment? Answer: yes Your guess: speaker
Question: Is it used for listening to music? Answer: yes Your guess: radio
Question: Is it portable? Answer: yes Your guess: mp3 player
Question: Is it battery-operated? Answer: yes Your guess: portable speaker
Question: Is it a type of speaker? Answer: no Your guess: mp3 player
Question: Is it a type of headphon

In [9]:
len(dataset)

120

In [7]:
# response = await call_gpt4(client, "Ask your first question", temperature=0.1, request_type="test", game_state=Game())

2024-07-05 22:28:28,485 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Tokens used: ChatCompletion(id='chatcmpl-9hh1QZRBB34qJnlHokBjgNTuzzJ6N', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Is the keyword a place?', role='assistant', function_call=None, tool_calls=None))], created=1720200508, model='gpt-4o-2024-05-13', object='chat.completion', service_tier=None, system_fingerprint='fp_ce0793330f', usage=CompletionUsage(completion_tokens=6, prompt_tokens=380, total_tokens=386))


In [4]:
game = Game(
    rounds=[
        Round(
            question="Is it a living object?",
            answer="yes"
        ),
        Round(
            question="Is it edible?",
            answer="yes"
        ),
        Round(
            question="Is it a fruit?",
            answer="no"
        ),
        Round(
            question="Is it an animal?",
            answer="no"
        ),
        Round(
            question="Is it a dairy product?",
            answer="no"
        ),
        Round(
            question="Is it a vegetable?",
            answer="yes"
        ),
        Round(
            question="Is it commonly green in color?",
            answer="no"
        ),
        Round(
            question="Is it commonly used in salads?",
            answer="no",
        ),
        Round(
            question="Is it commonly red in color?",
            answer="no",
        ),
        Round(
            question="Is it commonly cooked before eating?",
            answer="yes",
        ),
        # Round(
        #     question="Is it commonly used in soups?",
        #     answer="yes",
        #     guess="pumpkin",
        # ),
        Round(
            question="Is it commonly orange in color?",
            answer="yes",
            guess="carrot"
        )
    ]
)

In [5]:
candidates = await get_question_candidates(client, game)

2024-07-05 22:43:47,818 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Tokens used: 651


In [6]:
candidates

['Is it commonly used in soups or stews? (60% - 40%)',
 'Is it commonly used in desserts? (10% - 90%)',
 'Is it a root vegetable? (80% - 20%)',
 'Is it commonly used in baby food? (50% - 50%)',
 'Is it commonly used in Asian cuisine? (40% - 60%)',
 'Is it commonly used in Italian cuisine? (30% - 70%)',
 'Is it high in beta-carotene? (90% - 10%)',
 'Is it commonly used in juices? (20% - 80%)',
 'Is it commonly used in Thanksgiving dishes? (70% - 30%)',
 'Is it commonly used in French cuisine? (40% - 60%)']

In [138]:
candidates_scored = await get_question_candidates_scored(client, game, candidates)

2024-07-05 21:09:08,387 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [139]:
candidates_scored

['Is it a root vegetable? (90% - 10%)',
 'Is it commonly used in making purees? (70% - 30%)',
 'Is it commonly used in making baby food? (60% - 40%)',
 'Is it commonly used in making juices? (20% - 80%)',
 'Is it a tuber? (50% - 50%)',
 'Is it commonly used in making pies? (40% - 60%)',
 'Is it commonly used in making fries? (30% - 70%)',
 'Is it commonly used in making casseroles? (50% - 50%)',
 'Is it commonly used in making curries? (30% - 70%)',
 'Is it commonly used in stews? (80% - 20%)']

In [140]:
reasoning, question = await pick_question(client, game, candidates_scored)

2024-07-05 21:09:11,411 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [141]:
reasoning

'We know that the object is an edible, non-fruit, non-animal, non-dairy vegetable that is not commonly green or red, is commonly cooked before eating, and is commonly used in soups and is orange in color. Given these constraints, the vegetable is likely to be something like a carrot or a pumpkin. To narrow it down further, we should ask a question that splits the remaining possibilities effectively. The question "Is it a root vegetable?" is highly informative because it will help us distinguish between root vegetables like carrots and non-root vegetables like pumpkins. '

In [142]:
question

'Is it a root vegetable?'

In [143]:
reasoning, guessed_keyword = await guess(client, game)

2024-07-05 21:09:14,441 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [144]:
print(reasoning)

The keyword is a vegetable that is not green or red, is commonly cooked before eating, and is often used in soups. It is also commonly orange in color. Given these characteristics, and considering that both pumpkin and carrot have been guessed already, another common orange vegetable that fits these criteria is sweet potato. 


In [145]:
guessed_keyword

'sweet potato'

In [153]:
reasoning, answer_str = await answer(client, game, "carrot", "Is it commonly used in sex?")

2024-07-05 21:28:51,702 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [154]:
print(reasoning)

Carrots are not commonly associated with sexual use. While there might be rare instances or jokes about it, it is not a common or typical use for carrots. 


In [156]:
print(answer_str)

no
